# Importing Necessary Packages

In [1]:
# Needed for SQL Alchemy  
from sqlalchemy import create_engine
engine=create_engine('sqlite:///CourseData.db',echo=False)

# All query results are returned as Pandas DataFrames 
import pandas as pd

#Used for importing CSVs
import glob
import os

# Needed for %sql Magic
%load_ext sql

import sqlite3

# Creating the Databases

In [2]:
#Create CourseData and CourseDqtaWarehouse Databases
conn = sqlite3.connect("./CourseData.db")


In [3]:
# Initialize 
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

# Reading in the CSVs
- Combined each CSV into one DataFrame dropping necessary files 

In [4]:
CourseCatalog2017_2018=pd.read_csv("./SourceData/Catalogs/CourseCatalog2017_2018.csv")
CourseCatalog2018_2019=pd.read_csv("./SourceData/Catalogs/CourseCatalog2018_2019.csv")

In [5]:
#Reading and combining all course meetings CSVs

#Get list of folders/files in Source Data folder
folders=glob.glob('./SourceData/*')

#Delete folders/files from list that don't contain course meeting CSV files
del folders[5]
del folders[12]
del folders[16]

#Read CSVs from folders in list and combine into a single dataframe
all_files=[]
for path in folders:
    all_files += glob.glob(os.path.join(path, "course_meetings.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
course_meetings = pd.concat(df_from_each_file, ignore_index=True, sort=True)

course_meetings.head()

,crn,day,end,location,start,term
0,11701,W,2017-01-04T16:30:00,DSB 109,2017-01-04T08:30:00,Winter2017
1,11701,R,2017-01-05T16:30:00,DSB 109,2017-01-05T08:30:00,Winter2017
2,11701,F,2017-01-06T16:30:00,DSB 109,2017-01-06T08:30:00,Winter2017
3,11701,M,2017-01-09T16:30:00,DSB 109,2017-01-09T08:30:00,Winter2017
4,11701,T,2017-01-10T16:30:00,DSB 109,2017-01-10T08:30:00,Winter2017


In [6]:
#Reading and combining all courses CSVs

#Get list of folders/files in Source Data folder
folders=glob.glob('./SourceData/*')

#Delete folders/files from list that don't contain courses CSV files
del folders[5]
del folders[12]
del folders[16]

#Read CSVs from folders in list and combine into a single dataframe
all_files=[]
for path in folders:
    all_files += glob.glob(os.path.join(path, "courses.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
courses = pd.concat(df_from_each_file, ignore_index=True, sort=True)


#Split Meetings column in Courses DataFrame into separate columns
courses=pd.merge(courses, courses['meetings'].str.split(',',expand=True), left_index=True, right_index=True)

courses.head()

,act,cap,catalog_id,credits,crn,meetings,primary_instructor,rem,section,term,...,34,35,36,37,38,39,40,41,42,43
0,18,20,AH 0172,3.000,11705,[],Lauren Elise Cesiro,2,OL1,Winter2017,...,None,None,None,None,None,None,None,None,None,None
1,21,20,BI 0075,3.000,11706,[],Thomas B. Cunningham,-1,OL1,Winter2017,...,None,None,None,None,None,None,None,None,None,None
2,19,20,BI 0075,3.000,11724,[],Thomas B. Cunningham,1,OL2,Winter2017,...,None,None,None,None,None,None,None,None,None,None
3,20,25,BU 0211,3.000,11701,"[{'days': 'MTWRF', 'times': '0830am-0430pm', '...",Sharlene A. McEvoy,5,01,Winter2017,...,None,None,None,None,None,None,None,None,None,None
4,19,20,CO 0100,3.000,11707,[],David Gudelunas,1,OL1,Winter2017,...,None,None,None,None,None,None,None,None,None,None


# Course Catalog to SQL

In [7]:
CourseCatalog2017_2018.to_sql('CourseCatalog2017_2018',con=engine,if_exists='replace')
CourseCatalog2018_2019.to_sql('CourseCatalog2018_2019',con=engine,if_exists='replace')

CourseCatalog2018_2019.head()

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
0,AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,NaN,NaN,NaN,NaN,Students undertake an individualized program o...
1,AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,NaN,NaN,NaN,NaN,This seminar examines selected topics concerni...
2,BU,Business,BU 0211,Legal Environment of Business,3 Credits,Junior standing.,NaN,NaN,NaN,This course examines the broad philosophical a...
3,BU,Business,BU 0220,Environmental Law and Policy,3 Credits,NaN,NaN,NaN,"EVME Environmental Studies Major Elective, EVP...",This course surveys issues arising out of fede...
4,BU,Business,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,NaN,NaN,NaN,This course examines the components of common ...


# Course Meetings to SQL
- Used separate tables because DataFrame is too large to import all at once
- Sliced every 35,000 records in order to import

In [8]:
#import course_meetings DataFrame to SQL as separate tables because DataFrame is too large to import at once.
#Will use a union later in SQL to combine them

course_meetings[:35000].to_sql('course_meetings1',con=engine,if_exists='replace')
course_meetings[35000:70000].to_sql('course_meetings2',con=engine,if_exists='replace')
course_meetings[70000:105000].to_sql('course_meetings3',con=engine,if_exists='replace')
course_meetings[105000:140000].to_sql('course_meetings4',con=engine,if_exists='replace')
course_meetings[140000:175000].to_sql('course_meetings5',con=engine,if_exists='replace')
course_meetings[175000:210000].to_sql('course_meetings6',con=engine,if_exists='replace')
course_meetings[210000:245000].to_sql('course_meetings7',con=engine,if_exists='replace')
course_meetings[245000:280000].to_sql('course_meetings8',con=engine,if_exists='replace')
course_meetings[280000:315000].to_sql('course_meetings9',con=engine,if_exists='replace')
course_meetings[315000:].to_sql('course_meetings10',con=engine,if_exists='replace')

# Courses DataFrame to SQL
- Able to do once because the DataFrame is much smaller

In [9]:
courses.to_sql('courses',con=engine,if_exists='replace')

# Create Combined Course Catalog Table

In [13]:
%%sql

drop table if exists CourseCatalog;
Create table CourseCatalog(
program_code text,
catalog_id text PRIMARY KEY,
course_title text,
credits text,
prereqs text,
coreqs text,
fees text,
attributes text,
description text
);

insert into CourseCatalog
select distinct *
from (
select program_code,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description from CourseCatalog2017_2018
union all
select program_code,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description from CourseCatalog2018_2019
) a

 * sqlite:///CourseData.db
Done.
Done.
2220 rows affected.


[]

# Create Overall Course Meetings Table
- Specified column datatypes

In [14]:
%%sql

drop table if exists all_course_meetings;
CREATE TABLE all_course_meetings (
term text,
crn int,
location datexttatype,
day text,
start text,
end text
);


INSERT INTO all_course_meetings (term, crn, location, day, start, end)
select term,crn,location,day,start,end from course_meetings1
union all
select term,crn,location,day,start,end from course_meetings2
union all
select term,crn,location,day,start,end from course_meetings3
union all
select term,crn,location,day,start,end from course_meetings4
union all
select term,crn,location,day,start,end from course_meetings5
union all
select term,crn,location,day,start,end from course_meetings6
union all
select term,crn,location,day,start,end from course_meetings7
union all
select term,crn,location,day,start,end from course_meetings8
union all
select term,crn,location,day,start,end from course_meetings9
union all
select term,crn,location,day,start,end from course_meetings10
;

 * sqlite:///CourseData.db
Done.
Done.
280631 rows affected.


[]

# Alter table to break Start and End columns into separate Date and Time columns

In [15]:
%%sql

alter table all_course_meetings add MeetingDate date;
update all_course_meetings set MeetingDate = substr(start,1,10);

alter table all_course_meetings add StartTime time;
update all_course_meetings set StartTime=substr(start,12);

alter table all_course_meetings add EndTime time;
update all_course_meetings set EndTime=substr(end,12);


 * sqlite:///CourseData.db
Done.
280631 rows affected.
Done.
280631 rows affected.
Done.
280631 rows affected.


[]

# Create Cleaned Courses Table

- Broke courses with multiple meetings into separate records 
- Will allow for easier queries

In [16]:
%%sql

drop table if exists courses_clean;
CREATE TABLE courses_clean(
act BIGINT,
cap BIGINT,
catalog_id TEXT,
credits TEXT,
crn BIGINT,
meetings TEXT,
primary_instructor TEXT,
rem BIGINT,
section TEXT,
term TEXT,
title TEXT,
days TEXT,
times TEXT,
dates TEXT,
location TEXT
);



insert into courses_clean
SELECT * from
(
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[0] ,[1] ,[2] ,[3] 
    from courses
    where [4] is null 
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[4] ,[5] ,[6] ,[7]
    from courses
    where [4] is not null and [8] is null
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[8] ,[9] ,[10] ,[11]
    from courses
    where [8] is not null and [12] is null
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[12] ,[13] ,[14] ,[15]
    from courses
    where [12] is not null and [16] is null
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[16] ,[17] ,[18] ,[19]
    from courses
    where [16] is not null and [20] is null 
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[20] ,[21] ,[22] ,[23]
    from courses
    where [20] is not null and [24] is null 
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[24] ,[25] ,[26] ,[27]
    from courses
    where [24] is not null and [28] is null 
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[28] ,[29] ,[30] ,[31]
    from courses
    where [28] is not null and [32] is null 
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[32] ,[33] ,[34] ,[35]
    from courses
    where [32] is not null and [36] is null 
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[36] ,[37] ,[38] ,[39]
    from courses
    where [36] is not null and [40] is null 
union all
    select act ,cap ,catalog_id ,credits ,crn ,meetings ,primary_instructor ,rem ,section ,term ,title ,[40] ,[41] ,[42] ,[43]
    from courses
    where [40] is not null 
    ) 

 * sqlite:///CourseData.db
Done.
Done.
13860 rows affected.


[]

# Created Yet Another Pandas DataFrame from "Courses" data
- Wanted to clean up data with Python because SQL does not allow for easy character replacement (at least not this version of SQL)

In [17]:
courses_final=pd.read_sql_table('courses_clean',con=engine)


for i in ['[','{',':','days',"'",' ', ']']:
    courses_final['days']=courses_final['days'].str.replace(i,'')

for i in ['[','{',':','times',"'",' ']:
    courses_final['times']=courses_final['times'].str.replace(i,'')

for i in ['[','{',':','dates',"'",' ']:
    courses_final['dates']=courses_final['dates'].str.replace(i,'')

for i in ['[','{',':','location',"'",' ','}',']']:
    courses_final['location']=courses_final['location'].str.replace(i,'')
    
courses_final = courses_final.drop('meetings', 1)

courses_final.to_sql('courses_final',con=engine,if_exists='replace')

courses_final.head()

,act,cap,catalog_id,credits,crn,primary_instructor,rem,section,term,title,days,times,dates,location
0,18,20,AH 0172,3.000,11705,Lauren Elise Cesiro,2,OL1,Winter2017,History of Photography,,None,None,None
1,21,20,BI 0075,3.000,11706,Thomas B. Cunningham,-1,OL1,Winter2017,Ecology and Society,,None,None,None
2,19,20,BI 0075,3.000,11724,Thomas B. Cunningham,1,OL2,Winter2017,Ecology and Society,,None,None,None
3,20,25,BU 0211,3.000,11701,Sharlene A. McEvoy,5,01,Winter2017,Legal Environment of Business,MTWRF,0830am-0430pm,01/04-01/10,DSB109
4,19,20,CO 0100,3.000,11707,David Gudelunas,1,OL1,Winter2017,Human Communication Theories,,None,None,None


# Create new tables according to ERD

In [18]:
Calendar=pd.read_csv('Calendar.csv')
Calendar.to_sql('Calendar_nokey',con=engine,if_exists='replace')

#CourseCatalog table is already created

In [19]:
%%sql 

drop table if exists Terms;
Create table terms (
    Term varchar(50) PRIMARY KEY,
    StartDate date,
    EndDate date
);

insert into Terms
select term,min(MeetingDate) StartDate,max(MeetingDate) EndDate
from all_course_meetings
group by term;


drop table if exists Calendar;
Create table Calendar (
    CalendarDate date PRIMARY KEY,
    Weekday text
);

insert into Calendar
select CalendarDate,Weekday from Calendar_nokey;



drop table if exists CRN;
Create table CRN (
    CRN_Term int NOT NULL PRIMARY KEY,
    CRN text,
    Term text,
    Title text,
    Section text,
    Catalog_ID text
);

insert into CRN
select distinct (CRN||' '||Term) CRN_Term,CRN,Term,title,section,Catalog_ID from courses_final;



drop table if exists Course;
Create Table Course (
    act text,
    cap text,
    rem text,
    location text,
    CRN_Term text,
    Primary_Instructor text,
    StartTime time,
    EndTime time
);

insert into Course
select distinct act,cap,rem,a.location,(a.CRN||' '||a.Term) CRN_Term,Primary_Instructor,StartTime,EndTime
from 
all_course_meetings a
inner join
Courses_final b on a.crn=b.crn and a.location=b.location and a.term=b.term ;



 * sqlite:///CourseData.db
Done.
Done.
16 rows affected.
Done.
Done.
1734 rows affected.
Done.
Done.
13860 rows affected.
Done.
Done.
17 rows affected.


[]

In [20]:
conn.close()